In [2]:
import pandas as pd

df = pd.read_csv("../data/evaluation/all_dependencies.csv")

df_sample = df.sample(n=100)

df_sample.to_csv("../data/evaluation/test_dependencies.csv", index=False)

In [1]:
import json

data_file = "../data/evaluation/config5/all_dependencies_all.json"
output_file = "../data/evaluation/config5/all_dependencies_all_updated.json"

with open(data_file, "r", encoding="utf-8") as src:
    data = json.load(src)


for entry in data:
    context = entry["context"][:3]
    context_str = "\n\n".join([x["content"] for x in context])
    
    entry["context"] = context
    entry["context_str"] = context_str

with open(output_file, "a", encoding="utf-8") as dest:
    json.dump(data, dest, indent=2)


In [2]:
import json

data_file = "../data/evaluation/config5/all_dependencies_all.json"

with open(data_file, "r", encoding="utf-8") as src:
    data = json.load(src)

    print(len(data))

    print(data[0]["context"][0].keys())

    for x in data:
        context = x["context"]
        if len(context) == 0:
            print(x["index"])

500
dict_keys(['content', 'score', 'index', 'id'])


In [1]:
from prompt_settings import PrompSettingsFactory
from typing import Dict, List
from generator import GeneratorFactory, GeneratorEngine
from dotenv import load_dotenv
from util import load_config
from tqdm import tqdm
import backoff
import json


@backoff.on_exception(backoff.expo, Exception, max_tries=10)
def generate(generator: GeneratorEngine, messages: List) -> str:
    response = generator.generate(messages=messages)

    if not response:
        raise Exception("Response is empty.")

    return response


def run_generation(config: Dict, target_index: List[int]) -> None:
    
    print("Data file: ", config["data_file"])
    print("With RAG: ", config["with_rag"])
    

    with open(config["data_file"], "r", encoding="utf-8") as src:
        data = json.load(src)

    prompt_settings = PrompSettingsFactory.get_prompt_settings(tool_name=config["tool_name"])

    generator = GeneratorFactory().get_generator(
        model_name=config["model_name"], 
        temperature=config["temperature"]
    )

    for entry in tqdm(data, total=len(data), desc="Processing entries"):
        
        if not entry["index"] in target_index:
            continue


        if config["with_rag"]:
            query_str = prompt_settings.query_prompt.format(
                context_str=entry["context_str"], 
                task_str=entry["task_str"],
                format_str=prompt_settings.get_format_prompt()
            )
        else:
            query_str =f"{entry['task_str']}\n\n{prompt_settings.get_format_prompt()}"

        messages = [
            {
                "role": "system", 
                "content": entry["system_str"]
            },
            {
                "role": "user",
                "content": query_str
            }
        ]

        response = generate(
            generator=generator,
            messages=messages
        )

        #entry["responses"] = [response]
        entry["response"] = response
        
    output_file = config["data_file"]
    with open(output_file, "w", encoding="utf-8") as dest:
        json.dump(data, dest, indent=2)
            

config_file = "../generation_config.toml"
env_file = "../.env"

load_dotenv(dotenv_path=env_file)

# load config
config = load_config(config_file=config_file)
run_generation(config=config, target_index=[174, 267, 285])

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[07/18/24 08:19:40] INFO     PyTorch version 2.3.0 available.                                          ]8;id=981461;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/datasets/config.py\config.py]8;;\:]8;id=427141;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/datasets/config.py#58\58]8;;\

Data file:  ../data/evaluation/config3/all_dependencies_all_gpt-3.5-turbo-0125.json
With RAG:  True


[07/18/24 08:19:41] INFO     Initialize CfgNet prompt settings.                              ]8;id=577848;file:///home/simisimon/GitHub/cval/src/prompt_settings.py\prompt_settings.py]8;;\:]8;id=458454;file:///home/simisimon/GitHub/cval/src/prompt_settings.py#192\192]8;;\

                    INFO     GPT (gpt-3.5-turbo-0125) generator initialized.                        ]8;id=632280;file:///home/simisimon/GitHub/cval/src/generator.py\generator.py]8;;\:]8;id=453928;file:///home/simisimon/GitHub/cval/src/generator.py#47\47]8;;\

...

Processing entries:  35%|███▌      | 175/500 [00:02<00:05, 60.17it/s]

[07/18/24 08:19:47] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=591503;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=43931;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Processing entries:  54%|█████▎    | 268/500 [00:06<00:05, 38.73it/s]

[07/18/24 08:19:49] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=492976;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=595588;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Processing entries: 100%|██████████| 500/500 [00:08<00:00, 58.53it/s]
